# Create A Warped Disk Setup for RADMC-3D

## Imports

In [ ]:
import os
import subprocess

from matplotlib import cm
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

import numpy as np
from tqdm.auto import tqdm

from radmc3dPy.image import readImage, plotImage, makeImage
from astropy import constants as c

au = c.au.cgs.value
pc = c.pc.cgs.value
M_sun = c.M_sun.cgs.value
L_sun = c.L_sun.cgs.value
R_sun = c.R_sun.cgs.value
Grav = c.G.cgs.value
m_p = c.m_p.cgs.value

## Functions

In [ ]:
def logistic(a, r, r0, dr):
    return np.radians(a / (1.0 + np.exp((r - r0) / (0.1 * dr))))


def grid_refine_inner_edge(x_orig, nlev, nspan):
    x = x_orig.copy()
    rev = x[0] > x[1]
    for ilev in range(nlev):
        x_new = 0.5 * (x[1:nspan + 1] + x[:nspan])
        x_ref = np.hstack((x, x_new))
        x_ref.sort()
        x = x_ref
        if rev:
            x = x[::-1]
    return x

In [ ]:
def call_radmc(cmd, verbose=False, total=None):
    """
    Run radmc3d command and show progress bar instead.
    
    cmd : str
        the command to run, e.g. 'radmc3d mctherm'
        
    verbose : bool
        if True, then all output except the photon packges are shown
        if False, just the progress is shown.
        
    total : None | int
        total number of photon packages, if known
    """
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, text=True)
    output = []
    
    if 'nphot' in cmd:
        total = int(cmd.split('nphot')[1].split()[1])

    with tqdm(total=total, unit='photons') as pbar:
        for line in p.stdout:
            if 'Photon nr' in line:
                pbar.update(1000)
            elif verbose:
                print(line, end='')
            output += [line]
    rc = p.wait()
    return rc, ''.join(output)

In [ ]:
def warp_coordinates(x, y, z, phi, theta):
    xprime = x * np.cos(phi)   - y * np.sin(phi) * np.cos(theta) + z * np.sin(phi)   * np.sin(theta)
    yprime = x * np.sin(phi)   + y * np.cos(phi) * np.cos(theta) - z * np.sin(theta) * np.cos(phi)
    zprime = y * np.sin(theta) + z * np.cos(theta)
    return xprime, yprime, zprime

In [ ]:
def unwarp_coordinates(x, y, z, phi, theta):
    xprime =  x * np.cos(phi) + y * np.sin(phi)
    yprime = -x * np.sin(phi) * np.cos(theta) + y * np.cos(phi)   * np.cos(theta) + z * np.sin(theta)
    zprime =  x * np.sin(phi) * np.sin(theta) - y * np.sin(theta) * np.cos(phi)   + z * np.cos(theta)
    return xprime, yprime, zprime

## Setup 

### Set general parameters

In [ ]:
# Monte Carlo parameters
#
nphot_therm = 1000000
nphot_scat = 1000000

In [ ]:
# Grid parameters
#
n_r = 200
n_theta = 128
n_phi = 256
r_in = 0.5 * au
r_out = 100 * au
thetaup = 0.1       # Theta grid starting point (0=pole, but singular, so choose >0)

In [ ]:
# inner edge grid refinement
nlev_rin = 8
nspan_rin = 3

In [ ]:
# Disk parameters
#
sigma_0 = 1e1           # Gas surface density at 1 au [g / cm^2]
d2g = 0.01              # dust-to-gas ratio
gamma_gas = 1.0         # power-law exponent of the surface density
hor_0 = 0.05            # h/r at 1 au
hor_exp = 0.1           # flaring exponent

In [ ]:
# Star parameters
#
M_star = 2.4 * M_sun
R_star = 2.4 * R_sun
T_star = 1e4
star_coord = np.array([0., 0., 0.])

### Make the coordinates

In [ ]:
ri = np.geomspace(r_in, r_out, n_r + 1)
ri = grid_refine_inner_edge(ri, nlev_rin, nspan_rin)   # Refinement at inner edge
thetai = np.linspace(thetaup, np.pi - thetaup, n_theta + 1)
phii = np.linspace(0.0, 2.0 * np.pi, n_phi + 1)
rc = 0.5 * (ri[:-1] + ri[1:])
thetac = 0.5 * (thetai[:-1] + thetai[1:])
phic = 0.5 * (phii[:-1] + phii[1:])
n_r = len(rc)     # Recompute nr, because of refinement at inner edge

In [ ]:
# Make the full mesh
RC, THETAC, PHIC = np.meshgrid(rc, thetac, phic, indexing='ij')
XC = RC * np.sin(THETAC) * np.cos(PHIC)
YC = RC * np.sin(THETAC) * np.sin(PHIC)
ZC = RC * np.cos(THETAC)

### Make the warped model

In [ ]:
# un-warp the coordinates
warp = 60.
twist = 30.

warp_array = logistic(warp, rc, 40 * au, 30 * au)  # Specify the r0 and dr in AU
twist_array = logistic(twist, rc, 40 * au, 30 * au)

XU, YU, ZU = unwarp_coordinates(
    XC, YC, ZC,
    warp_array[:, None, None],
    twist_array[:, None, None])

RU = np.sqrt(XU**2 + YU**2 + ZU**2)
THETAU = np.pi / 2.0 - np.arctan2(ZU, np.sqrt(XU**2 + YU**2))
PHIU = (np.arctan2(YU, XU) + 2 * np.pi)%(2 * np.pi)

In [ ]:
# Make the dust density model
sig_g = sigma_0 * (RU / au)**-gamma_gas
H = hor_0 * (RU / au)**hor_exp * RU
rho_g = (sig_g / (np.sqrt(2. * np.pi) * H)) * np.exp(-(ZU**2 / H**2) / 2.0)
rho_d = d2g * rho_g

In [ ]:
# Make the velocity model
# this velocity is the original r,theta,phi velocity in the un-warped state,
# before warping to our coordinates
# wasn't yet really computed
VRU = np.zeros_like(RU)
VTU = np.zeros_like(RU)
VPU = np.sqrt(Grav * M_star / RU)

vturb = 0.001 * VPU

# make cartesian velocities

VXU = VRU * np.sin(THETAU) * np.cos(PHIU) + VTU * np.cos(THETAU) * np.cos(PHIU) - VPU * np.sin(THETAU) * np.sin(PHIU)
VYU = VRU * np.sin(THETAU) * np.sin(PHIU) + VTU * np.cos(THETAU) * np.sin(PHIU) + VPU * np.sin(THETAU) * np.cos(PHIU)
VZU = VRU * np.cos(THETAU)                - VTU * np.sin(THETAU)

# warp velocities
VXW, VYW, VZW = warp_coordinates(
    VXU, VYU, VZU,
    warp_array[:, None, None],
    twist_array[:, None, None])

VRW =  VXW * np.sin(THETAC) * np.cos(PHIC) + VYW * np.sin(THETAC) * np.sin(PHIC) + VZW * np.cos(THETAC)
VTW = -VXW * np.sin(THETAC) * np.sin(PHIC) + VYW * np.sin(THETAC) * np.cos(PHIC)
VPW =  VXW * np.cos(THETAC) * np.cos(PHIC) + VYW * np.cos(THETAC) * np.sin(PHIC) - VZW * np.sin(THETAC)

In [ ]:
ir = 100
it = int(n_theta // 2)
s = 1

f, ax = plt.subplots(subplot_kw={'projection':'3d'}, figsize=(12, 12))
_r = 3
ax.set_xlim(-_r, _r)
ax.set_ylim(-_r, _r)
ax.set_zlim(-_r, _r)

ax.view_init(elev=5, azim=120)

#for ir in range(0, len(rc), s):
ax.plot3D(XC[ir, it, :] / au, YC[ir, it, :] / au, ZC[ir, it, :] / au, 'k', lw=0.5)
ax.plot3D(XU[ir, it, :] / au, YU[ir, it, :] / au, ZU[ir, it, :] / au, 'r', lw=0.5)

scale = 5000000

arr_u = np.array([[XU, XU + scale * VXU], [YU, YU + scale * VYU], [ZU, ZU + scale * VZU]])
arr_w = np.array([[XC, XC + scale * VXW], [YC, YC + scale * VYW], [ZC, ZC + scale * VZW]])

for iphi in range(n_phi):
    ax.plot3D(*arr_u[:, :, ir, it, iphi]/au, 'r', lw=0.5)
    ax.plot3D(*arr_w[:, :, ir, it, iphi]/au, 'k', lw=0.5)

### Write RADMC3D setup

In [ ]:
# Write the wavelength_micron.inp file
#
lam1 = 0.1e0
lam2 = 7.0e0
lam3 = 25.e0
lam4 = 1.0e4
n12 = 20
n23 = 100
n34 = 30
lam12 = np.geomspace(lam1, lam2, n12, endpoint=False)
lam23 = np.geomspace(lam2, lam3, n23, endpoint=False)
lam34 = np.geomspace(lam3, lam4, n34, endpoint=True)
lam = np.concatenate([lam12, lam23, lam34])
nlam = lam.size

In [ ]:
# Write the wavelength file
#
with open('wavelength_micron.inp', 'w+') as f:
    f.write('%d\n' % (nlam))
    for value in lam:
        f.write('%13.6e\n' % (value))

In [ ]:
# Write the stars.inp file
#
with open('stars.inp', 'w+') as f:
    f.write('2\n')
    f.write('1 %d\n\n' % (nlam))
    f.write('%13.6e %13.6e %13.6e %13.6e %13.6e\n\n' % (R_star, M_star, *star_coord))
    for value in lam:
        f.write('%13.6e\n' % (value))
    f.write('\n%13.6e\n' % (-T_star))

In [ ]:
# Write the grid file
#
with open('amr_grid.inp', 'w+') as f:
    f.write('1\n')                       # iformat
    f.write('0\n')                       # AMR grid style  (0=regular grid, no AMR)
    f.write('100\n')                     # Coordinate system: spherical
    f.write('0\n')                       # gridinfo
    f.write('1 1 1\n')                   # Include r,theta coordinates
    f.write('%d %d %d\n' % (n_r, n_theta, n_phi))  # Size of grid
    for value in ri:
        f.write('%13.6e\n' % (value))      # X coordinates (cell walls)
    for value in thetai:
        f.write('%13.6e\n' % (value))      # Y coordinates (cell walls)
    for value in phii:
        f.write('%13.6e\n' % (value))      # Z coordinates (cell walls)

In [ ]:
# Write the density file
#
with open('dust_density.inp', 'w+') as f:
    f.write('1\n')                       # Format number
    f.write('%d\n' % (n_r * n_theta * n_phi))     # Nr of cells
    f.write('1\n')                       # Nr of dust species
    data = rho_d.ravel(order='F')         # Create a 1-D view, fortran-style indexing
    data.tofile(f, sep='\n', format="%13.6e")
    f.write('\n')

In [ ]:
# Dust opacity control file
#
with open('dustopac.inp', 'w+') as f:
    f.write('2               Format number of this file\n')
    f.write('1               Nr of dust species\n')
    f.write('============================================================================\n')
    f.write('1               Way in which this dust species is read\n')
    f.write('0               0=Thermal grain\n')
    f.write('silicate        Extension of name of dustkappa_***.inp file\n')
    f.write('----------------------------------------------------------------------------\n')

In [ ]:
# Write the molecule number density file.
#
CO_abundance = 1e-4
fact_CO = CO_abundance / (2.3 * m_p)
nco = rho_g * fact_CO
with open('numberdens_co.inp', 'w+') as f:
    f.write('1\n')                       # Format number
    f.write('%d\n' % (n_r * n_theta * n_phi))     # Nr of cells
    data = nco.ravel(order='F')          # Create a 1-D view, fortran-style indexing
    data.tofile(f, sep='\n', format="%13.6e")
    f.write('\n')

In [ ]:
# Write the gas velocity field
#
with open('gas_velocity.inp', 'w+') as f:
    f.write('1\n')                       # Format number
    f.write('%d\n' % (n_r * n_theta * n_phi))     # Nr of cells
    for iphi in range(n_phi):
        for itheta in range(n_theta):
            for ir in range(n_r):
                f.write('%13.6e %13.6e %13.6e\n' % (VRW[ir, itheta, iphi], VTW[ir, itheta, iphi], VPW[ir, itheta, iphi]))

In [ ]:
# Write the microturbulence file
#
with open('microturbulence.inp', 'w+') as f:
    f.write('1\n')                       # Format number
    f.write('%d\n' % (n_r * n_theta * n_phi))     # Nr of cells
    data = vturb.ravel(order='F')        # Create a 1-D view, fortran-style indexing
    data.tofile(f, sep='\n', format="%13.6e")
    f.write('\n')

In [ ]:
# Write the lines.inp control file
#
with open('lines.inp', 'w') as f:
    f.write('1\n')
    f.write('1\n')
    f.write('co    leiden    0    0\n')

In [ ]:
# Write the radmc3d.inp control file
#
with open('radmc3d.inp', 'w+') as f:
    f.write('nphot = %d\n' % (nphot_therm))
    f.write('nphot_scat = %d\n' % (nphot_scat))
    f.write('scattering_mode_max = 1\n')
    f.write('iranfreqmode = 1\n')
    f.write('tgas_eq_tdust = 1\n')

## Run RADMC-3D

General image parameters

In [ ]:
dpc         = 140.     # Distance in parsec (for conversion to Jy/pixel in 1.3 mm map)
incl        = 45.
phi         = 45.
npix        = 200
sizeau      = 100
lamda_image = 1.3e3

### Thermal Monte Carlo

In [ ]:
rc, output = call_radmc('radmc3d mctherm setthreads 8', total=nphot_therm)

### Channel Map

In [ ]:
# Now let's make a set of channel maps
vkms = 0
rc, output = call_radmc(f'radmc3d image imolspec 1 iline 2 vkms {vkms} incl {incl} phi {phi} npix {npix} setthreads 8 sizeau {sizeau}', total=nphot_scat)

Read and plot the image

In [ ]:
im = readImage()
f, ax = plt.subplots(figsize=(10, 8))
plotImage(im, au=True, cmap=cm.hot, ax=ax)

In [ ]:
f, ax = plt.subplots()
vmax = im.image.max()
ax.imshow(im.image[:,:,0], norm=LogNorm(vmin=1e-2 * vmax, vmax=vmax), interpolation='none', extent=np.array([-r_out, r_out, -r_out, r_out])/au)
ax.set_xlim(-100,100)
ax.set_ylim(-100,100);

### Continuum image

In [ ]:
rc, output = call_radmc(f'radmc3d image dpc {dpc} incl {incl} phi {phi} lambda {lamda_image} sizeau {sizeau} npix {npix} setthreads 8', total=nphot_scat)

In [ ]:
im_mm = readImage()
f, ax = plt.subplots(figsize=(10, 8))
plotImage(im_mm, au=True, log=True, maxlog=3, bunit='inu', dpc=dpc, cmap='magma', ax=ax);

In [ ]:
plt.show()